In [15]:
import random
import os
import shutil
import glob
import gc
import sys
import h5py
import time
import datetime
import pickle
import librosa
import warnings
import matplotlib.pyplot as plt

import numpy as np
from numpy import array
import pandas as pd
from pandas.plotting import parallel_coordinates
from pydub import AudioSegment

#imported for testing
import wave
import contextlib

# for outputing file
from scipy.cluster.vq import vq, kmeans, whiten
import scipy.stats.stats as st

from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.datasets.samples_generator import make_blobs
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.metrics import auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.utils import class_weight

import keras

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten, Add, Dropout, Input, Activation
from keras.layers import TimeDistributed, Bidirectional, LSTM, LeakyReLU
from keras.models import Sequential
from keras import optimizers, regularizers
from keras.utils import np_utils, to_categorical

#from tensorflow.python.client import device_lib
from keras import backend
import tensorflow as tf


In [27]:
# load training and testing ... 

condition = 'reverb'

new_train = 'D://Datasets//TRAINING//padded_deamplified_homenoised_reverberated//train//' + condition + '//'
new_test = 'D://Datasets//TRAINING//padded_deamplified_homenoised_reverberated//test//' + condition + '//'
new_val = 'D://Datasets//TRAINING//padded_deamplified_homenoised_reverberated//val//' + condition + '//'

def load_vectors(path):
    files = sorted(os.listdir(path))
    X = np.expand_dims(np.zeros((48, 272)), axis=0)
    y = []
    for npy in files:
        current = np.load(path+npy)
        X = np.vstack((X, current))
        label = [files.index(npy)]*len(current)       
        y = y + label       
    X = X[1:]
    y = to_categorical(y)    
    print(X.shape)
    print(y.shape)    
    return X, y
    
X_train, y_train = load_vectors(new_train)
X_val, y_val = load_vectors(new_val)
X_test, y_test = load_vectors(new_test)

(5370, 48, 272)
(5370, 5)
(1792, 48, 272)
(1792, 5)
(1626, 48, 272)
(1626, 5)


In [21]:
def mil_squared_error(y_true, y_pred):
    return tf.keras.backend.square(tf.keras.backend.max(y_pred) - tf.keras.backend.max(y_true))

adam = tf.keras.optimizers.Adam(learning_rate=1e-5)

def create_cnn(num_layers, n_neurons, n_batch, nbindex, dropout, classes, dense_layers):

    model = Sequential()

    model.add(Convolution1D(nb_filter=nbindex, filter_length=fillength, activation = 'relu',
                            input_shape=(X_train.shape[1], X_train.shape[2]), kernel_constraint=maxnorm(3)))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))

    model.add(Convolution1D(nb_filter=nbindex*2, filter_length=fillength, activation = 'relu',
                            kernel_constraint=maxnorm(3)))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))

    model.add(Convolution1D(nb_filter=nbindex*2, filter_length=fillength, activation = 'relu',
                            kernel_constraint=maxnorm(3)))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))

    model.add(Convolution1D(nb_filter=nbindex, filter_length=fillength, activation = 'relu',
                            kernel_constraint=maxnorm(3)))  
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))
    
    model.add(Flatten())
    
    for i in range(0, dense_layers):
        model.add(Dense(n_neurons, activation = 'relu'))
        model.add(Dropout(dropout))
        
    # Here, get the empirical class mean and covariance of training samples

    model.add(Dense(classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=adam,
                  metrics=['accuracy', mil_squared_error])

    return model

In [22]:
sample_rate = 44100
frame_number = 48
hop_length = 441  # frame size= 2 * hop
segment_length = int(sample_rate * 0.2)  # 0.2
segment_pad = int(sample_rate * 0.02)     # 0.02
overlapping = int(sample_rate * 0.1)   # 0.1

classes = 5
NumofFeaturetoUse = 272
n_neurons = 1024 * 4
dense_layers = 3
num_layers = 4
fillength = 3
nbindex = 1024 * 3
dropout = 0.2
n_batch = 128
n_epoch = 100

In [23]:
model = create_cnn(num_layers=num_layers, 
                   n_neurons=n_neurons, 
                   n_batch=n_batch, 
                   nbindex=nbindex, 
                   dropout=dropout, 
                   classes=classes, 
                   dense_layers=dense_layers)

C:\Users\Ash Gao\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", input_shape=(48, 272), kernel_constraint=<keras.con..., filters=3072, kernel_size=3)`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Ash Gao\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_constraint=<keras.con..., filters=6144, kernel_size=3)`
  app.launch_new_instance()
C:\Users\Ash Gao\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_constraint=<keras.con..., filters=6144, kernel_size=3)`
C:\Users\Ash Gao\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_constraint=<keras.con..., filters=3072, kernel_size=3)`


In [ ]:
print("Fit model on training data")
history = model.fit(
    X_train,
    y_train,
    batch_size=n_batch,
    epochs=n_epoch,
    validation_data=(X_val, y_val), verbose=1
)

Fit model on training data
Train on 5370 samples, validate on 1792 samples
Epoch 1/100


In [ ]:
#from sklearn.metrics import f1_score

y_preds = [np.argmax(val) for val in model.predict(X_test)]
y_trues = [np.argmax(val) for val in y_test]
print(accuracy_score(y_trues, y_preds))

model.save('models//cnn.hdf5')

return model